In [24]:
import jovian
import requests
from bs4 import BeautifulSoup
import pandas as pd
from PIL import Image
from pytesseract import pytesseract
from collections import defaultdict

In [25]:
header = {'User_Agent': 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36'}

# This is the books information crawling section

In [26]:
bookKey = ["Title",
                "Author",
                "About_author",
                "Foreword",
                "Description",
                "Image",
                "Genres", 
                "Series",
                "Order",
                "Pages",
                "Format",
                "First_published",
                "Rating",
                "Review_count"]


In [27]:
books = []
reviews_link = []
series_all = []
authors_link = []
genres_link = []
bookFile = open('links1.txt', "r")
lines = bookFile.readlines()

count = 0
for line in lines:
    book = {key: None for key in bookKey}
    count += 1
    print("Line{}: {}".format(count, line.strip()))
    # download the web page using requests
    book_info = line.strip()
    response = requests.get(book_info,headers=header)

    # capture the text content of the page from response
    page_contents = response.text

    # Save the page as html to avoid scrapping multiple times while experimenting
    with open('books.html', 'w', encoding="utf-8") as f:
        f.write(page_contents)

    # read it html page to scrape information
    with open('books.html', 'r', encoding="utf-8") as f:
        html_source = f.read()

    # The BeautifulSoup returns a document which we shall use to extract information
    doc_book = BeautifulSoup(html_source, 'html.parser')

    for br in doc_book.select("br"):
        br.replace_with("\n")
    
    # get all the company name in the page
    book_name = doc_book.find('h1', attrs={'data-testid': 'bookTitle'})
    #print(book_name)
    book['Title'] = book_name.text.strip()
    # for the first company name, get the exact name

    book_author = doc_book.find('span', attrs={'data-testid': 'name'})
    #print(book_author)
    book['Author'] = book_author.text.strip()
    # for the first company name, get the exact name

    book_rating = doc_book.find('div', class_= 'RatingStatistics__rating')
    #print(book_rating)
    book['Rating'] = float(book_rating.text.strip())

    review_count = doc_book.find('span', attrs={'data-testid': 'reviewsCount'})
    #print(review_count)
    review_count = review_count.text.strip()
    review_count = ''.join(x for x in review_count if x.isdigit())
    book['Review_count'] = int(review_count)
    # for the first company name, get the exact name

    book_description = doc_book.find('div', attrs={'data-testid': 'description'})
    #print(book_description)
    br = book_description.find('b')
    if br != None:
        book['Foreword'] = br.text.strip()
        book_description.find('b').decompose()

    book_description = doc_book.find('div', attrs={'data-testid': 'description'})
    des = book_description.text.strip()
    book['Description'] = des

    book_image = doc_book.find('img', class_='ResponsiveImage')
    book['Image'] = book_image['src']

    genres = []
    for type in doc_book.find_all('span', class_='BookPageMetadataSection__genreButton'):
        genre = type.text.strip()
        a = type.find('a')
        i = a.get('href')
        genres_link.append(i)
        genres.append(genre)

    genre = ", ".join(genres)
    book['Genres'] = genre

    book_series = doc_book.find('h3', class_='Text Text__title3 Text__italic Text__regular Text__subdued')
    if book_series != None:
        a = book_series.find('a')
        series_link = a.get('href')
        series_all.append(series_link)
        series = book_series.text.strip().split('#')
        book['Series'] = series[0]
        if len(series) != 1:
            book['Order'] = series[1]

    book_form = doc_book.find('p', attrs={'data-testid': 'pagesFormat'})
    #print(book_form)
    book_form = book_form.text.strip()
    book_form = book_form.split(' ')
    book['Pages'] = int(book_form[0])
    book['Format'] = book_form[2]
    # for the first company name, get the exact name

    book_date = doc_book.find('p', attrs={'data-testid': 'publicationInfo'})
    book_date = book_date.text.strip()
    book_date = book_date.split(' ')
    book_date.remove('First')
    book_date.remove('published')
    book['First_published'] = ' '.join(book_date)

    author_info = doc_book.find('div', class_='FeaturedPerson__infoPrimary')
    author_info = author_info.find('a')
    author_info = author_info.get('href')
    authors_link.append(author_info)

    reviews = []
    reviews = doc_book.find_all('span', class_='Text Text__body3')
    for i in reviews:
        review = i.find('a')
        if review is None:
            continue
        else:
            review = str(review.get('href'))
            reviews_link.append(review)

    author_des = doc_book.find_all('div', class_='DetailsLayoutRightParagraph__widthConstrained')
    author_des = author_des[1]
    author_des = author_des.text.strip()
    book['About_author'] = author_des
    print(book)
    books.append(book)

Line1: https://www.goodreads.com/book/show/2767052-the-hunger-games
{'Title': 'The Hunger Games', 'Author': 'Suzanne Collins', 'About_author': 'Librarian Note: There is more than one author in the Goodreads database with this name.\n\nSince 1991, Suzanne Collins has been busy writing for children’s television. She has worked on the staffs of several Nickelodeon shows, including the Emmy-nominated hit Clarissa Explains it All and The Mystery Files of Shelby Woo. For preschool viewers, she penned multiple stories for the Emmy-nominated Little Bear and Oswald. She also co-wrote the critically acclaimed Rankin/Bass Christmas special, Santa, Baby! Most recently she was the Head Writer for Scholastic Entertainment’s Clifford’s Puppy Days.\n\nWhile working on a Kids WB show called Generation O! she met children’s author James Proimos, who talked her into giving children’s books a try.\n\nThinking one day about Alice in Wonderland, she was struck by how pastoral the setting must seem to kids w

# add links of author, series, reviews and genres to file 

In [28]:
reviewFile = open('reviews.txt', "a") 
for i in reviews_link:
    reviewFile.write(i+"\n")        
reviewFile.close()

seriesFile = open('series.txt', "a")
for i in series_all:
    seriesFile.write(i+"\n")        
seriesFile.close()

authorFile = open("authors.txt", "a") 
for i in authors_link:
    authorFile.write(i+"\n")        
authorFile.close()

genreFile = open('genres.txt', "a")
for i in genres_link:
    genreFile.write(i+"\n")        
genreFile.close()

# start crawling authors information

In [29]:
authorKey = ["Name",
            "Born",
            "Died",
            "Hometown",
            "Biography",
            "Image",
            "Genre",
            "Website",
            "Influences",  
            "Rating",
            "Rating_count",
            "Review_count",
            "Works",          
            "Typical_series"
]

In [30]:
authors = []
authorFile = open('authors.txt', "r")
lines = authorFile.readlines()
lines = list(set(lines))

count = 0
for line in lines:
    author = {key: None for key in authorKey}
    count += 1
    print("Line{}: {}".format(count, line.strip()))
    # download the web page using requests
    author_info = line.strip()
    response = requests.get(author_info,headers=header)

    # capture the text content of the page from response
    page_contents = response.text

    # Save the page as html to avoid scrapping multiple times while experimenting
    with open('author.html', 'w', encoding="utf-8") as file:
        file.write(page_contents)

    # read it html page to scrape information
    with open('author.html', 'r', encoding="utf-8") as file:
        html_source = file.read()

    # The BeautifulSoup returns a document which we shall use to extract information
    doc_author = BeautifulSoup(html_source, 'html.parser')

    for br in doc_author.select("br"):
        br.replace_with("\n")

    authorName = doc_author.find('h1', class_='authorName')
    author['Name'] = authorName.text.strip()

    author_hometown = doc_author.find('div', class_='dataTitle').next_sibling
    author['Hometown'] = author_hometown.text.strip()

    data = doc_author.find_all('div', class_='dataTitle')
    for i in data:
        info = i.text.strip()
    
        if info == 'Website':
            i = i.find_next_sibling('div')     
            author[info] = i.find('a').get('href')
        elif info == 'Influences':
            i = i.find_next_sibling('div')
            a = i.find('span', attrs={'style':'display:none'}) 
            if a != None:
                author[info] = a.text.strip()
            else:
                author[info] = i.text.strip()
        else:
            i = i.find_next_sibling('div')
            attr = i['class']     
            if attr == ['dataItem']:    
                author[info] = i.text.strip()
    
    author_img = doc_author.find('div', class_='leftContainer authorLeftContainer')
    author['Image'] = author_img.find('img').get('src')

    author_info = doc_author.find('div', class_='aboutAuthorInfo').find('span')
    author['Biography'] = author_info.text.strip()

    author_data = doc_author.find('div', class_='hreview-aggregate')
    for i in author_data.find_all('span'):
        check = i.get('itemprop')
        if check == 'ratingValue':
            author['Rating'] = i.text.strip()
        elif check == 'ratingCount':
            author['Rating_count'] = i.text.strip()
        elif check == 'reviewCount':
            author['Review_count'] = i.text.strip()
            author_work = i.find_next('a')
            author_book = author_work.get('href')
            author['Works'] = author_work.text.strip().split()[0]

    author['Typical_series'] = []
    typicalSeries = doc_author.find_all('div', class_='seriesDesc')
    for series in typicalSeries:
        series = series.find('a', class_='bookTitle')
        series = series.text.strip()
        author['Typical_series'].extend([series])

    print(author)
    authors.append(author)


Line1: https://www.goodreads.com/author/show/1825.Harper_Lee
{'Name': 'Harper Lee', 'Born': 'April 28, 1926', 'Died': 'February 19, 2016', 'Hometown': 'in Monroeville, Alabama, The United States', 'Biography': 'Harper Lee, known as Nelle, was born in the Alabama town of Monroeville, the youngest of four children of Amasa Coleman Lee and Frances Cunningham Finch Lee. Her father, a former newspaper editor and proprietor, was a lawyer who served on the state legislature from 1926 to 1938. As a child, Lee was a tomboy and a precocious reader, and enjoyed the friendship of her schoolmate and neighbor, the young Truman Capote.\n\nAfter graduating from high school in Monroeville, Lee enrolled at the all-female Huntingdon College in Montgomery (1944-45), and then pursued a law degree at the University of Alabama (1945-50), pledging the Chi Omega sorority. While there, she wrote for several student publications and spent a year as editor of the campus humor ma', 'Image': 'https://images.gr-asse

# start crawling genres

In [31]:
keyList = ["Name",
            "Details"]

In [32]:
genres = []
genreFile = open('genres.txt', "r")
lines = genreFile.readlines()
lines = list(set(lines))

count = 0
for line in lines:
    genre = {key: None for key in keyList}
    count += 1
    print("Line{}: {}".format(count, line.strip()))
    # download the web page using requests
    genre_info = line.strip()
    response = requests.get(genre_info,headers=header)

    # capture the text content of the page from response
    page_contents = response.text

    # Save the page as html to avoid scrapping multiple times while experimenting
    with open('genre.html', 'w', encoding="utf-8") as f:
        f.write(page_contents)

    # read it html page to scrape information
    with open('genre.html', 'r', encoding="utf-8") as f:
        html_source = f.read()

    # The BeautifulSoup returns a document which we shall use to extract information
    doc_genre = BeautifulSoup(html_source, 'html.parser')

    name = doc_genre.find('div', class_='genreHeader')
    genre['Name']= name.text.strip()

    details = doc_genre.find('div', class_='mediumText reviewText')
    if details != None:
        for br in details.select("br"):
            br.replace_with("\n")
        content = details.find('span', attrs={'style':'display:none'})
        if content is None:
            genre['Details'] = details.text.strip()
        else:
            genre['Details'] = content.text.strip()

    print(genre)
    genres.append(genre)


Line1: https://www.goodreads.com/genres/fantasy
{'Name': 'Fantasy', 'Details': 'Fantasy is a genre that uses magic and other supernatural forms as a primary element of plot, theme, and/or setting. Fantasy is generally distinguished from science fiction and horror by the expectation that it steers clear of technological and macabre themes, respectively, though there is a great deal of overlap between the three (collectively known as speculative fiction or science fiction/fantasy)\n\nIn its broadest sense, fantasy comprises works by many writers, artists, filmmakers, and musicians, from ancient myths and legends to many recent works embraced by a wide audience today, including young adults, most of whom are represented by the works below.'}
Line2: https://www.goodreads.com/genres/dystopia
{'Name': 'Dystopia', 'Details': 'Dystopia is a form of literature that explores social and political structures. It is a creation of a nightmare world - unlike its opposite, Utopia, which is an ideal wo

# start crawling series

In [33]:
keyList = ["Name",
            "Information"]

In [34]:
allSeries = []
seriesFile = open('series.txt', "r")
lines = seriesFile.readlines()
lines = list(set(lines))

count = 0
for line in lines:
    series = {key: None for key in keyList}
    count += 1
    print("Line{}: {}".format(count, line.strip()))
    # download the web page using requests
    series_info = line.strip()
    response = requests.get(series_info,headers=header)

    # capture the text content of the page from response
    page_contents = response.text

    # Save the page as html to avoid scrapping multiple times while experimenting
    with open('series.html', 'w', encoding="utf-8") as f:
        f.write(page_contents)

    # read it html page to scrape information
    with open('series.html', 'r', encoding="utf-8") as f:
        html_source = f.read()

    # The BeautifulSoup returns a document which we shall use to extract information
    doc_series = BeautifulSoup(html_source, 'html.parser')

    name = doc_series.find('div', class_='responsiveSeriesHeader__title')
    name = name.text.strip()
    if 'Series' in name:
        name = name.strip('Series')
    series['Name'] = name

    information = doc_series.find_all('div', attrs={'class':'u-paddingBottomSmall'})
    series['Information'] = information[1].text.strip()
    print(series)
    allSeries.append(series)


Line1: https://www.goodreads.com/series/73758-the-hunger-games
{'Name': 'The Hunger Games ', 'Information': 'The thrilling trilogy expertly described by Suzanne Collins. You enter a future world where every year two tributes from each district are thrown into an arena expected to fight to the death or get knocked out by other means. Only one can survive.Also see: The Hunger Games Companion booksSeries also known as:* Die Tribute von Panem [German]* Nälkäp…More'}
Line2: https://www.goodreads.com/series/255474-to-kill-a-mockingbird
{'Name': 'To Kill a Mockingbird ', 'Information': ''}
Line3: https://www.goodreads.com/series/45175-harry-potter
{'Name': 'Harry Potter ', 'Information': 'Orphan Harry learns he is a wizard on his 11th birthday when Hagrid escorts him to magic-teaching Hogwarts School. As a baby, his mother\'s love protected him and vanquished the villain Voldemort, leaving the child famous as "The Boy who Lived." With his friends Hermione and Ron, Harry has to defeat the retu

# start crawling reviews

In [35]:
keyList = ["Username",
            "Book",
            "Date",
            "Rating",
            "Comment"]

In [36]:
reviews = []
reviewFile = open('reviews.txt', "r")
lines = reviewFile.readlines()
lines = list(set(lines))


In [37]:
count = 0
for line in lines:
    review = {key: None for key in keyList}
    count += 1
    print("Line{}: {}".format(count, line.strip()))
    # download the web page using requests
    review_info = line.strip()
    if review_info == 'None':
        continue
    else:
        response = requests.get(review_info,headers=header)

        # capture the text content of the page from response
        page_contents = response.text

        # Save the page as html to avoid scrapping multiple times while experimenting
        with open('review.html', 'w', encoding="utf-8") as f:
            f.write(page_contents)

        # read it html page to scrape information
        with open('review.html', 'r', encoding="utf-8") as f:
            html_source = f.read()

        # The BeautifulSoup returns a document which we shall use to extract information
        doc_review = BeautifulSoup(html_source, 'html.parser')

        for br in doc_review.select("br"):
            br.replace_with("\n")
        book_name = doc_review.find('a', class_='bookTitle')
        if book_name != None:
            review['Book'] = book_name.text.strip().split('(')[0]

        username = doc_review.find('a', class_='userReview')
        if username != None:
            review['Username'] = username.text.strip()

        date = doc_review.find('span', attrs={'itemprop':'datePublished'})
        if date != None:
            review['Date'] = date.text.strip()

        star = doc_review.find('meta', attrs={'itemprop':'ratingValue'})
        if star != None:
            review['Rating'] = star.get('content')

        comment = doc_review.find('div', attrs={'itemprop':'reviewBody'})
        if comment != None:
            review['Comment'] = comment.text.strip()

        print(review)
        reviews.append(review)

Line1: https://www.goodreads.com/review/show/3727144341
{'Username': 'Tharindu Dissanayake', 'Book': 'Harry Potter and the Order of the Phoenix ', 'Date': 'Jan 05, 2021', 'Rating': '5', 'Comment': '"Galloping gargoyles!"\n\n\n"Hang on. This Lord Voldything\'s back, you say."\n\n\nWhen a book gets too long, there\'s always the risk of giving up halfway, or start skimming through parts. I think, The Order of Phoenix qualifies as long (the version I read had around a 1000 pages), but I wish it to be even longer. Only Rowling can do that. I didn\'t find one boring bit, or anything out of place. We are reaching a near-perfect level of writing here at the fifth of the series, which, in my opinion, is an excellent concoction of suspense, hope, triumph, disappointment, humor, anger, happiness, sadness and so many others.\n\n\n"I\'ve always wanted to try firewhisky -"\n\n\nRight from the beginning, you\'ll find everything to be intriguing as always, with an excellent set of new characters being

KeyboardInterrupt: 